In [6]:
import wandb
import tempfile
import os
import sys
import yaml

In [2]:
sys.path.append('../covidcf/')

In [14]:
from config.base import ConfigSchema, Config

In [4]:
api = wandb.Api()
run = api.run("rhacking/covid-cf/u2f8yh84")

In [9]:
with tempfile.TemporaryDirectory() as tmpdirname:
    for a in run.logged_artifacts():
        if a.name.startswith('base_config'):
            artifact_path = a.download(tmpdirname)
            config_path = next(os.scandir(artifact_path)).path
            with open(config_path, 'r') as f:
                config = yaml.load(f, Loader=yaml.SafeLoader)
            config = ConfigSchema.load(config)

In [10]:
config

Config(dataset=DatasetConfig(name='rumc', target=<DatasetTarget.PCR: 'pcr'>, validation=ValidationConfig(cv=RepeatedKFoldEvaluationConfig(n_folds=10, n_repeats=1), metric='roc_auc'), test=TestConfig(cv=PartEvaluationConfig(part='test'), metrics=['accuracy', 'roc_auc']), should_aggregate=True, use_clinical_features=True, use_visual_features=True, feature_value_fraction_required=0.16), pipeline=PipelineConfig(imputation=ImputationConfig(categorical=<ImputationType.CONSTANT: 'constant'>, numeric=<ImputationType.MEAN: 'mean'>), feature_selection=FeatureSelectionConfig(fs_type=<FeatureSelectionType.NONE: 'none'>, num_features=None, threshold=None), estimator_type=<EstimatorType.LR: 'lr'>, n_bagging=None), parameters={'estimator': {'solver': 'liblinear'}})

In [11]:
with tempfile.TemporaryDirectory() as tmpdirname:
    for a in run.logged_artifacts():
        if a.name.startswith('sweep_config'):
            artifact_path = a.download(tmpdirname)
            config_path = next(os.scandir(artifact_path)).path
            with open(config_path, 'r') as f:
                sweep_config = yaml.load(f, Loader=yaml.SafeLoader)

In [28]:
import copy
def update_base_config(base_config: Config, update):
    config_dict = ConfigSchema.dump(copy.deepcopy(base_config))
    clinical_features = []
    visual_features = []
    for k, v in update.items():
        keys = k.split('.')
        if k.startswith('use_'):
            if v:
                if k.startswith('use_Clinical'):
                    clinical_features.append(k.replace('use_Clinical_', ''))
                else:
                    visual_features.append(k.replace('use_Visual_', ''))
        elif len(keys) > 1:
            current = config_dict
            for key in keys[:-1]:
                current = current[key]
            current[keys[-1]] = v
        else:
            config_dict['parameters']['estimator'][k] = v
            
    if len(clinical_features) > 0:
        config_dict['dataset']['use_clinical_features'] = clinical_features
    
    if len(visual_features) > 0:
        config_dict['dataset']['use_visual_features'] = visual_features
    
    return config_dict

In [29]:
update_base_config(config, run.config)

{'dataset': {'target': 'PCR',
  'should_aggregate': True,
  'feature_value_fraction_required': 0.2957474331964979,
  'validation': {'metric': 'roc_auc', 'cv': {'n_repeats': 1, 'n_folds': 10}},
  'use_clinical_features': ['PH',
   'CRP',
   'EAL',
   'LDH',
   'MCH',
   'PO2',
   'RWT',
   'TSH',
   'sex',
   'LEUK',
   'EIWIT',
   'UREUM',
   'KALIUM',
   'LVIDDI',
   'LVRW78',
   'EAM6447',
   'EEAM162',
   'FOSFAAT',
   'GLUCOSE',
   'KETONEN',
   'LAVO666',
   "LEUCO'S",
   'LVMA364',
   'LVMA367',
   'LVMA383',
   'LVOTVTI',
   'NATRIUM',
   'ANIONGAP',
   'GAMMA GT',
   'LAVOLA2C',
   'LAVOLA4C',
   'LVMASS2D',
   'FERRITINE',
   'BACTERIËN',
   'KREATININE',
   'LAAA2CSIMP',
   'LVOT PGMAX',
   'LVOTPGMEAN',
   'LYMFOCYTEN',
   'URINEKWEEK',
   'BASE EXCESS',
   'CHOLESTEROL',
   'ERYTROCYTEN',
   'HEMATOCRIET',
   'TROMBOCYTEN',
   'ALBUMINE LKC',
   'KOOLMONOXIDE',
   'MONOCYTEN MD',
   'ALGEMENE KWEEK',
   'METHEMOGLOBINE',
   'ZUURSTOFFRACTIE',
   'LEUCOCYTEN URINE',
   'SPIR

In [15]:
config

Config(dataset=DatasetConfig(name='rumc', target=<DatasetTarget.PCR: 'pcr'>, validation=ValidationConfig(cv=RepeatedKFoldEvaluationConfig(n_folds=10, n_repeats=1), metric='roc_auc'), test=TestConfig(cv=PartEvaluationConfig(part='test'), metrics=['accuracy', 'roc_auc']), should_aggregate=True, use_clinical_features=True, use_visual_features=True, feature_value_fraction_required=0.16), pipeline=PipelineConfig(imputation=ImputationConfig(categorical=<ImputationType.CONSTANT: 'constant'>, numeric=<ImputationType.MEAN: 'mean'>), feature_selection=FeatureSelectionConfig(fs_type=<FeatureSelectionType.NONE: 'none'>, num_features=None, threshold=None), estimator_type=<EstimatorType.LR: 'lr'>, n_bagging=None), parameters={'estimator': {'solver': 'liblinear'}})

In [13]:
run.config

{'C': 0.16287118210587914,
 'penalty': 'l2',
 'use_Clinical_CK': False,
 'use_Clinical_EA': False,
 'use_Clinical_PH': True,
 'use_Clinical_PT': False,
 'use_Clinical_CRP': True,
 'use_Clinical_EAL': True,
 'use_Clinical_LDH': True,
 'use_Clinical_MCH': True,
 'use_Clinical_MCV': False,
 'use_Clinical_PO2': True,
 'use_Clinical_RWT': True,
 'use_Clinical_TSH': True,
 'use_Clinical_age': False,
 'use_Clinical_sex': True,
 'use_Clinical_ALAT': False,
 'use_Clinical_ASAT': False,
 'use_Clinical_EEAL': False,
 'use_Clinical_HCO3': False,
 'use_Clinical_LEUK': True,
 'use_Clinical_MCHC': False,
 'use_Clinical_PCO2': False,
 'use_Clinical_TDIE': False,
 'use_Clinical_BILIR': False,
 'use_Clinical_EIWIT': True,
 'use_Clinical_MVEPT': False,
 'use_Clinical_UREUM': True,
 'use_Clinical_AVVMAX': False,
 'use_Clinical_KALIUM': True,
 'use_Clinical_LVIDDI': True,
 'use_Clinical_LVRW78': True,
 'use_Clinical_AVPGMAX': False,
 'use_Clinical_CALCIUM': False,
 'use_Clinical_EAM6447': True,
 'use_Clini

In [14]:
a = list(run.logged_artifacts())[0]

In [15]:
a= run.logged_artifacts().next()

In [16]:
a.download()

'.\\artifacts\\meta_config-v1'

In [25]:
!pip install --upgrade wandb

  Attempting uninstall: wandb
    Found existing installation: wandb 0.10.32
    Uninstalling wandb-0.10.32:
      Successfully uninstalled wandb-0.10.32


You should consider upgrading via the 'c:\users\roel\.virtualenvs\diagcovid-ckc2berq\scripts\python.exe -m pip install --upgrade pip' command.
